In [49]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score

from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score
from scipy.stats import norm
from sklearn.metrics import confusion_matrix, plot_roc_curve, make_scorer, balanced_accuracy_score



from scipy.stats import ttest_ind
import pandas as pd

In [13]:
import pickle as pickle

3. На наборе данных BRCA_pam50 построить модель случайноголеса, имеющую сбалансированную точность больше 0.8 (задача —выбрать оптимальные параметры). Для поиска оптимальныхпараметров и оценки точности использовать функциюGridSearchCV с опцией cv=RepeatedStratifiedKFold(n_repeats=10).

In [14]:
data = pd.read_csv('https://raw.githubusercontent.com/s-a-nersisyan/HSE_bioinformatics_2021/master/seminar17/BRCA_pam50.tsv', sep='\t', index_col=0)

In [50]:
data = pd.read_csv('BRCA_pam50.tsv', sep='\t', index_col=0)


le = LabelEncoder()
data['Subtype'] = le.fit_transform(data['Subtype'])

model = RandomForestClassifier()

X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:-1], data['Subtype'])

params = {#'max_depth': np.arange(3, 10), 
          'n_estimators': [150, 170, 200, 220, 250],
         'max_features': ['auto', 'sqrt', 'log2'],
         'bootstrap': ['True', 'False']}



grid = GridSearchCV(model, params, scoring=make_scorer(balanced_accuracy_score), cv=RepeatedStratifiedKFold(n_repeats=10))

grid.fit(X_train, y_train)

best_model = grid.best_estimator_
preds = best_model.predict(X_test)
print(balanced_accuracy_score(y_test, preds))
print(grid.best_params_)


0.8008446163765314
{'bootstrap': 'False', 'max_features': 'auto', 'n_estimators': 220}


4. Построить две модели линейного SVM для классификациирецидива рака молочной железы (bc_data.pkl, bc_ann.pkl). Перваямодель должна быть построена с использованием всех генов,вторая — на 11 генах (файл genes_SVM.txt). Вывести качествамоделей (TPR, TNR, ROC AUC) на обучающей и тестовойвыборках. Интерпретировать результат.

In [19]:
with open("bc_data.pkl", "rb") as data, open("bc_ann.pkl", "rb") as ann:
    df = pickle.load(data)
    ann = pickle.load(ann)
    
# в файле аннотации индекс = строке основного датасета + в столбце тип элемента
    
selected_genes = "TRIP13;UBE2C;ZWINT;EPN3;KIF4A;ECHDC2;MTFR1;STARD13;IGFBP6;NUMA1;CCNL2".split(";")



X_train_trimmed = df.loc[ann.loc[ann["Dataset type"] == "Training"].index, selected_genes].to_numpy()
# 11 генов из тестовой выборки 

X_train = df.loc[ann.loc[ann["Dataset type"] == "Training"].index].to_numpy()
y_train = ann.loc[ann["Dataset type"] == "Training", "Class"].to_numpy()


X_test_trimmed = df.loc[ann.loc[ann["Dataset type"] == "Validation"].index, selected_genes].to_numpy()
X_test = df.loc[ann.loc[ann["Dataset type"] == "Validation"].index].to_numpy()
y_test = ann.loc[ann["Dataset type"] == "Validation", "Class"].to_numpy()

# 
model11 = SVC(kernel="linear")
model = SVC(kernel="linear")

model.fit(X_train, y_train) #  целая и снизу обрезанная гавна
model11.fit(X_train_trimmed, y_train)

models = [model, model11]
labels = ["all genes", "11 genes"]
dataset_types = ["validation", "training"]

for curr_model, label in zip(models, labels):
    test_data = [X_test_trimmed, X_train_trimmed] if label == "11 genes" else [X_test, X_train]
    y = [y_test, y_train]
    
    for dataset_type, curr_X, curr_y in zip(dataset_types, test_data, y):
        
        print(f"SVC for {label}, {dataset_type} dataset:")
        curr_preds = curr_model.predict(curr_X)
        print("ROC-AUC: ", roc_auc_score(curr_y, curr_preds))
        curr_conf_matrix = confusion_matrix(curr_y, curr_preds)
        curr_tpr = curr_conf_matrix[0][0] / (curr_conf_matrix[0][0] + curr_conf_matrix[0][1])
        curr_tnr = curr_conf_matrix[1][1] / (curr_conf_matrix[1][1] + curr_conf_matrix[1][0])
        print("TPR: ", curr_tpr)
        print("TNR: ", curr_tnr)

SVC for all genes, validation dataset:
ROC-AUC:  0.6041666666666666
TPR:  0.75
TNR:  0.4583333333333333
SVC for all genes, training dataset:
ROC-AUC:  1.0
TPR:  1.0
TNR:  1.0
SVC for 11 genes, validation dataset:
ROC-AUC:  0.7291666666666666
TPR:  0.75
TNR:  0.7083333333333334
SVC for 11 genes, training dataset:
ROC-AUC:  0.7246642246642248
TPR:  0.8461538461538461
TNR:  0.6031746031746031


5. Определить гены, средняя экспрессия которых статистическизначимо различается между обучающей и тестовой выборками(bc_data.pkl, bc_ann.pkl, использовать𝑡-критерий Стьюдента).Какова доля этих генов среди всех?

In [65]:
with open("bc_data.pkl", "rb") as data, open("bc_ann.pkl", "rb") as ann:
    df = pickle.load(data)
    ann = pickle.load(ann)
    
a = df.shape[1]

X_train = df.loc[ann.loc[ann["Dataset type"] == "Training"].index]
X_test = df.loc[ann.loc[ann["Dataset type"] == "Validation"].index]


df.loc['pval'] = ttest_ind(X_train, X_test)[1]
df.sort_values(by='pval', axis=1)
df = df.T


In [66]:
df

,GSM107072,GSM107073,GSM107074,GSM107075,GSM107076,GSM107077,GSM107078,GSM107079,GSM107083,GSM107084,...,GSM441912,GSM441913,GSM441914,GSM441915,GSM441916,GSM441917,GSM441918,GSM441919,GSM441921,pval
A1CF,6.171391,6.580377,6.941675,6.523469,6.592214,6.761344,6.558884,6.626959,6.624572,6.597635,...,6.719660,7.372394,7.036898,6.757148,6.505351,6.592653,6.568996,6.484071,6.914977,5.753293e-01
A2M,10.848897,9.739404,9.989207,11.243880,11.302001,9.253217,10.676338,11.296706,10.917747,10.393812,...,10.470359,10.492195,10.037092,11.529877,10.185212,10.861886,10.836349,10.984289,10.000197,1.867661e-01
AACS,7.777530,8.172784,8.175175,7.286647,7.628476,9.178121,7.829007,7.477054,8.435515,8.726798,...,7.401514,7.659186,7.867834,8.108791,8.866148,7.908883,7.652979,6.841230,8.092569,1.815174e-03
AAGAB,7.606607,7.859401,7.485449,7.695029,6.610815,7.399232,7.430664,7.361514,7.881902,7.952224,...,7.236209,6.739222,7.315677,7.301934,7.533166,7.140476,7.055975,7.233096,7.433678,1.026696e-01
AAK1,6.394179,6.304567,6.215530,7.172579,6.417622,6.386813,6.652547,5.944220,6.015653,6.374048,...,6.580730,5.695605,6.082650,6.466926,6.764427,6.514528,6.658431,7.306780,7.998971,1.118005e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW10,7.356916,7.808439,7.750089,7.453132,7.606603,7.672605,7.407300,7.603397,7.678944,7.452117,...,7.606002,7.950501,7.646302,7.650898,7.576646,7.449552,7.542041,7.377068,7.317501,1.815445e-01
ZWILCH,6.417884,7.170811,5.951289,6.192658,6.096380,7.359414,6.064417,6.116630,6.725369,5.619477,...,6.941322,6.328662,6.225161,6.096737,6.485330,5.871643,5.948556,6.893088,6.355751,6.714115e-04
ZWINT,7.614747,8.820814,8.746304,7.719506,8.469398,9.786675,7.765456,7.697013,8.100966,8.956603,...,8.702793,8.495653,8.204738,7.607147,9.136435,8.315122,6.937566,8.880852,8.738591,1.581438e-02
ZYX,9.248246,9.134095,9.316072,9.237652,9.456770,8.600402,9.570348,9.477195,9.473874,9.463023,...,9.432843,8.825105,8.981991,9.365254,8.548642,8.878892,9.035272,9.195416,8.915722,3.830581e-05


In [69]:
df[df['pval'] < 0.05].shape[0] / a * 100

60.04895645452203

7762